# 5강) BERT를 활용한 Dense Passage Retrieval 실습
*   KorQuAD train 데이터셋을 활용해서 Dense Encoder을 학습시킬 수 있다.

*   Dense Encoder를 통해 Dense Embedding을 만들 수 있다.

*   Dense Embedding을 활용하여 passage retrieval를 진행할 수 있다.


### Requirements

In [1]:
!pip install datasets==2.7.0 -q
!pip install transformers==4.24.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.6/451.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.9 MB/s eta 0:00:00


## 데이터셋 로딩


KorQuAD train 데이터셋을 search corpus 및 학습 데이터로 활용

In [2]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

9606

## 토크나이저 준비 - Huggingface 제공 tokenizer 이용

BERT를 encoder로 사용하므로, hugginface에서 제공하는 "bert-base-multilingual-cased" tokenizer를 활용

In [3]:
from transformers import AutoTokenizer
import numpy as np

model_checkpoint = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [4]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [7]:
print(corpus[0])
tokenized_input = tokenizer(corpus[0], padding="max_length", truncation=True)
tokenizer.decode(tokenized_input['input_ids'])

이스마일파의 또 다른 분파는 북아프리카에서 아랍인과 베르베르족 사이의 긴장을 이용해 권력을 잡고 파티마 왕조(909-1171)를 세운 우바이둘라 알마흐디(909-934) 세력이다. 이들은 969년 이집트를 정복하고 옛 수도 푸스타트 근처에 까히라(al-Qahirah 승리자; 오늘날의 카이로)라는 도시를 세웠다. 칼리파조는 까히라에 거대한 도서관을 짓고 이슬람 교리의 본산이자 세계에서 가장 오래된 대학이라는 알아즈하르(al-Azhar) 성원(聖院)을 만들었다. 이 칼리파조는 11세기 초 국력을 키워 팔레스타인과 시리아 등지를 세력권에 넣었으며, 바그다드의 칼리파도 한때는 카이로의 세력에 맥을 못추었다. 그러나 1171년 아이유브 왕조의 살라흐 앗 딘(1138-1193; 살라딘)에 의해 멸망했으며 이집트와 시리아는 다시 수니파의 손으로 들어갔다.


'[CLS] 이스마일파의 또 다른 분파는 북아프리카에서 아랍인과 베르베르족 사이의 긴장을 이용해 권력을 잡고 파티마 왕조 ( 909 - 1171 ) 를 세운 우바이둘라 알마흐디 ( 909 - 934 ) 세력이다. 이들은 969년 이집트를 정복하고 옛 수도 푸스타트 근처에 까히라 ( al - Qahirah 승리자 ; 오늘날의 카이로 ) 라는 도시를 세웠다. 칼리파조는 까히라에 거대한 도서관을 짓고 이슬람 교리의 본산이자 세계에서 가장 오래된 대학이라는 알아즈하르 ( al - Azhar ) 성원 ( 聖 院 ) 을 만들었다. 이 칼리파조는 11세기 초 국력을 키워 팔레스타인과 시리아 등지를 세력권에 넣었으며, 바그다드의 칼리파도 한때는 카이로의 세력에 맥을 못추었다. 그러나 1171년 아이유브 왕조의 살라흐 앗 딘 ( 1138 - 1193 ; 살라딘 ) 에 의해 멸망했으며 이집트와 시리아는 다시 수니파의 손으로 들어갔다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

## Dense encoder (BERT) 학습 시키기

HuggingFace BERT를 활용하여 question encoder, passage encoder 학습

In [8]:
from tqdm import tqdm, trange
import argparse
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup

torch.manual_seed(2023)
torch.cuda.manual_seed(2023)
np.random.seed(2023)
random.seed(2023)

1) Training Dataset 준비하기 (question, passage pairs)

---



In [9]:
# Use subset (10 example) of original training dataset
sample_idx = np.random.choice(range(len(dataset['train'])), 20)
training_dataset = dataset['train'][sample_idx]

Negative sampling을 위한 negative sample들을 샘플링

In [10]:
# set number of neagative sample
num_neg = 3

corpus = np.array(corpus)
p_with_neg = []

for c in training_dataset['context']:
  while True:
    neg_idxs = np.random.randint(len(corpus), size=num_neg)

    if not c in corpus[neg_idxs]:
      p_neg = corpus[neg_idxs]

      p_with_neg.append(c)
      p_with_neg.extend(p_neg)
      break


In [11]:
print('[Positive context]')
print(p_with_neg[0], '\n')
print('[Negative context]')
print(p_with_neg[1], '\n', p_with_neg[2])

[Positive context]
2008년 7월, 국가기록원과 뉴라이트 전국연합에서 대통령 기록물을 사사로이 봉하 마을로 옮긴 건에 대하여 검찰에 불법적인 '무단 유출'로 기록물에 관련된 전 비서관과 행정관들을 고발하여 수사가 진행되었다. 기록원의 고발 조치에 대해 노무현 측 비서관인 김경수는 "청와대와 정부의 목적이 기록 회수가 아닌 참여정부 흠집 내기였음이 분명해진 것"이라고 밝히며 "참모진과 향후 대응 방안을 논의하겠다"라고 말했다. 검찰이 대통령 기록물 유출 실체 규명에 나서게 됨에 따라 신·구 정권 간 대립과 갈등이 격해질 것이라는 시각도 있다. 한편 회수 조치를 하는 와중에 기록이 담긴 하드디스크(데이터)뿐만 아니라 노무현이 개인 자금으로 구매한 e-지원 시스템 서버(하드웨어)까지 반환하라고 요구했고, 노무현 측은 개인 재산이라는 이유로 거부했다. 이때 대통령기록물 관리에 관한 법률 제17조 5항에 따르면, 전직 대통령 및 전직 대통령이 지정한 대리인은 대통령 기록물을 열람할 수 있고, 그것이 비밀로 지정되어 있지 않다면, 그것을 출판하거나 언론매체에 공표할 수 있다고 규정되어 있다(다만 이 조항은 2010년 2월 4일 개정되었다). 그러나 법률 내용에도 불구하고 외부로 반출하여 지정되지 않은 장소에 보관하는 것은 기록물의 관리 및 보안상 유출 우려가 있어 인정할 수 없다는 것이 법학자들의 다수 견해가 있었다. 

[Negative context]
청나라는 1639년(숭덕 4년)부터 1643년(숭덕 8년)에 걸쳐 흑룡강 일대 원주민들(예벤키족, 다우르족 등)에 대한 군사행동을 실시하여 그들을 복속시켰다. 1643년 겨울에 바실리 콜랴코프의 코사크 기병대가 흑룡강 유역에 나타나기 시작했고, 1650년대에는 예로페이 하바로프와 아무르 원주민들 사이의 충돌이 일어났다. 루스 차르국은 1655년 "아무르 지역 군사총독"을 임명했다. 이렇게 러시아와의 국지적 충돌이 계속되자 순치제 때 청나라는 러시아와 국경을 마주한 조선에 러시아와 싸울 군사를 요청하였다.

In [12]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(p_with_neg, padding="max_length", truncation=True, return_tensors='pt')


In [13]:
max_len = p_seqs['input_ids'].size(-1)
p_seqs['input_ids'] = p_seqs['input_ids'].view(-1, num_neg+1, max_len)
p_seqs['attention_mask'] = p_seqs['attention_mask'].view(-1, num_neg+1, max_len)
p_seqs['token_type_ids'] = p_seqs['token_type_ids'].view(-1, num_neg+1, max_len)

print(p_seqs['input_ids'].size())  #(num_example, pos + neg, max_len)

torch.Size([20, 4, 512])


In [14]:
train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'],
                              q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

2) BERT encoder 학습시키기

BertEncoder 모델 정의 후, question encoder, passage encoder에 pre-trained weight 불러오기

In [15]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()

  def forward(self, input_ids,
              attention_mask=None, token_type_ids=None):

      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)

      pooled_output = outputs[1]
      return pooled_output

In [16]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.transform.dense.bias', 'cls.predictions.

Train function 정의 후, 두개의 encoder fine-tuning 하기


In [17]:
def train(args, num_neg, dataset, p_model, q_model):

  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

  # Optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
        {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in q_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

  # Start training!
  global_step = 0

  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()

  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()

      targets = torch.zeros(args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)
        targets = targets.cuda()

      p_inputs = {'input_ids': batch[0].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'attention_mask': batch[1].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'token_type_ids': batch[2].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1)
                  }

      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}

      p_outputs = p_model(**p_inputs)  #(batch_size*(num_neg+1), emb_dim)
      q_outputs = q_model(**q_inputs)  #(batch_size*, emb_dim)

      # Calculate similarity score & loss
      p_outputs = p_outputs.view(args.per_device_train_batch_size, -1, num_neg+1)
      q_outputs = q_outputs.view(args.per_device_train_batch_size, 1, -1)

      sim_scores = torch.bmm(q_outputs, p_outputs).squeeze()  #(batch_size, num_neg+1)
      sim_scores = sim_scores.view(args.per_device_train_batch_size, -1)
      sim_scores = F.log_softmax(sim_scores, dim=1)

      loss = F.nll_loss(sim_scores, targets)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1

      torch.cuda.empty_cache()



  return p_model, q_model

In [18]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01
)

In [19]:
p_encoder, q_encoder = train(args, num_neg, train_dataset, p_encoder, q_encoder)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch: 100%|██████████| 2/2 [00:22<00:00, 11.29s/it]


## Dense Embedding을 활용하여 passage retrieval 실습해보기

In [20]:

valid_corpus = list(set([example['context'] for example in dataset['validation']]))[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

print(query)
print(ground_truth, '\n\n')

# valid_corpus

불법행위 당시에는 예견할 수 없었던 새로운 손해 또는 확대된 손해의 예는?
민법 766조 1항 소정의 손해 및 가해자를 안다고 하는 경우에 손해발생 사실을 안다는 것은 단순히 손해발생의 사실만을 안 때라는 뜻이 아니고 가해행위가 불법행위로써 이를 원인으로 하여 손해배상을 소구할 수 있다는 사실을 안 때 이자, 가해행위와 손해의 발생 사이에 인과관계가 있으며 위법하고 과실이 있는 것까지도 안 때이다. 가해행위와 이로 인한 현실적인 손해의 발생 사이에 시간적 간격이 있는 불법행위에 기한 손해배상채권의 경우, 소멸시효의 기산점이 되는 “불법행위를 한 날"의 의미는 단지 관념적이고 부동적인 상태에서 잠재적으로만 존재하고 있는 손해가 그 후 현실화되었다고 볼 수 있는 때, 다시 말하자면 손해의 결과발생이 현실적인 것으로 되었다고 할 수 있을 때 이고, 후유증 등으로 인하여 불법행위 당시에는 예견할 수 없었던 새로운 손해가 발생하였다거나 예상외로 손해가 확대된 경우에 있어서는 그러한 사유가 판명되었을 때 비로소 새로이 발생 또는 확대된 손해를 알았다고 보아야 한다. 다만, 현실화된 손해의 정도나 액수까지 구체적으로 알아야 하는 것은 아니다. 




앞서 학습한 passage encoder, question encoder을 이용해 dense embedding 생성

In [21]:
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [22]:
with torch.no_grad():
  p_encoder.eval()
  q_encoder.eval()

  q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

  p_embs = []
  for p in valid_corpus:
    p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
    p_emb = p_encoder(**p).to('cpu').numpy()
    p_embs.append(p_emb)

p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

print(p_embs.size(), q_emb.size())

torch.Size([11, 768]) torch.Size([1, 768])


<ipython-input-22-6e141c8b7ed9>:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)


생성된 embedding에 dot product를 수행 => Document들의 similarity ranking을 구함

In [23]:
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size())

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(dot_prod_scores)
print(rank)

torch.Size([1, 11])
tensor([[17.7813, 18.9347, 13.6126,  5.2556, 13.3107, 12.0886, 13.6584, 15.2902,
         12.9865, 16.0539, 19.2158]])
tensor([10,  1,  0,  9,  7,  6,  2,  4,  8,  5,  3])


Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [24]:
k = 5
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
  print(valid_corpus[rank[i]])

[Search query]
 불법행위 당시에는 예견할 수 없었던 새로운 손해 또는 확대된 손해의 예는? 

[Ground truth passage]
민법 766조 1항 소정의 손해 및 가해자를 안다고 하는 경우에 손해발생 사실을 안다는 것은 단순히 손해발생의 사실만을 안 때라는 뜻이 아니고 가해행위가 불법행위로써 이를 원인으로 하여 손해배상을 소구할 수 있다는 사실을 안 때 이자, 가해행위와 손해의 발생 사이에 인과관계가 있으며 위법하고 과실이 있는 것까지도 안 때이다. 가해행위와 이로 인한 현실적인 손해의 발생 사이에 시간적 간격이 있는 불법행위에 기한 손해배상채권의 경우, 소멸시효의 기산점이 되는 “불법행위를 한 날"의 의미는 단지 관념적이고 부동적인 상태에서 잠재적으로만 존재하고 있는 손해가 그 후 현실화되었다고 볼 수 있는 때, 다시 말하자면 손해의 결과발생이 현실적인 것으로 되었다고 할 수 있을 때 이고, 후유증 등으로 인하여 불법행위 당시에는 예견할 수 없었던 새로운 손해가 발생하였다거나 예상외로 손해가 확대된 경우에 있어서는 그러한 사유가 판명되었을 때 비로소 새로이 발생 또는 확대된 손해를 알았다고 보아야 한다. 다만, 현실화된 손해의 정도나 액수까지 구체적으로 알아야 하는 것은 아니다. 

Top-1 passage with score 19.2158
민법 766조 1항 소정의 손해 및 가해자를 안다고 하는 경우에 손해발생 사실을 안다는 것은 단순히 손해발생의 사실만을 안 때라는 뜻이 아니고 가해행위가 불법행위로써 이를 원인으로 하여 손해배상을 소구할 수 있다는 사실을 안 때 이자, 가해행위와 손해의 발생 사이에 인과관계가 있으며 위법하고 과실이 있는 것까지도 안 때이다. 가해행위와 이로 인한 현실적인 손해의 발생 사이에 시간적 간격이 있는 불법행위에 기한 손해배상채권의 경우, 소멸시효의 기산점이 되는 “불법행위를 한 날"의 의미는 단지 관념적이고 부동적인 상태에서 잠재적으로만 존재하고 있는 손해가 그 후 현실화되었다고 볼 수 있는 때, 다시 말하

### **콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다. 모델 라이선스 : MIT License

